In [1]:
import requests
import pandas as pd
import numpy as np
import json
import time
import re
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display

# --- CONFIGURATION ---
FETCH_ALL_PAGES = True   # Set True to scan ALL markets (takes 2-3 mins)
PROFIT_MIN = 0.5         # Minimum profit % to display (0.5%)
PROFIT_MAX = 20.0        # Maximum profit % (Filters out fake 99% errors)

# --- PART 1: HELPER FUNCTIONS ---

def extract_numbers(text):
    """
    Extracts critical numbers (e.g., 2025, 95.5, 300) from titles.
    Ignores single digits (1, 2) to avoid false negatives on generic terms.
    """
    if not isinstance(text, str): return set()
    # Matches integers > 9 and all decimals (e.g., "0.5", "10.5")
    nums = re.findall(r'\b\d{2,}\b|\d+\.\d+', text.replace(',', ''))
    return set(float(n) for n in nums)

def get_polymarket_data():
    """Fetches active Polymarket events with full order book parsing."""
    print("🔵 [1/3] Fetching Polymarket Data...")
    base_url = "https://gamma-api.polymarket.com/events"
    limit = 100
    offset = 0
    all_rows = []
    
    while True:
        params = {'active': 'true', 'closed': 'false', 'limit': limit, 'offset': offset}
        try:
            response = requests.get(base_url, params=params)
            data = response.json()
            if not data: break
            
            for event in data:
                if not event.get('markets'): continue
                for m in event['markets']:
                    try:
                        # Extract outcomes and prices safely
                        outs = json.loads(m.get('outcomes', '[]'))
                        pxs = json.loads(m.get('outcomePrices', '[]'))
                        
                        if not pxs or len(pxs) != len(outs): continue
                        
                        # Store outcomes mapping
                        row_outcomes = {o: float(p) for o, p in zip(outs, pxs)}
                        
                        all_rows.append({
                            "Platform": "Polymarket",
                            "Title": event.get('title', 'N/A'),
                            "Outcomes": row_outcomes
                        })
                    except: continue
            
            offset += limit
            if not FETCH_ALL_PAGES and offset >= 200: break # Safety limit for testing
            time.sleep(0.1)
            
        except Exception as e:
            print(f"   ⚠️ Poly Error: {e}")
            break
            
    print(f"   ✅ Loaded {len(all_rows)} Polymarket events.")
    return pd.DataFrame(all_rows)

def get_kalshi_data():
    """Fetches active Kalshi events and calculates Ask prices."""
    print("🟢 [2/3] Fetching Kalshi Data...")
    base_url = "https://api.elections.kalshi.com/trade-api/v2/events"
    limit = 100
    cursor = None
    all_rows = []
    
    while True:
        params = {'limit': limit, 'status': 'open', 'with_nested_markets': 'true'}
        if cursor: params['cursor'] = cursor
            
        try:
            response = requests.get(base_url, params=params)
            data = response.json()
            events = data.get('events', [])
            cursor = data.get('cursor')
            
            if not events: break
                
            for event in events:
                for m in event.get('markets', []):
                    # We need the Ask price (Cost to Buy)
                    yes_ask = m.get('yes_ask')
                    no_ask = m.get('no_ask')
                    
                    if yes_ask is None or no_ask is None: continue
                    
                    # Convert cents to dollars
                    row_outcomes = {
                        "Yes": yes_ask / 100.0,
                        "No": no_ask / 100.0
                    }
                    
                    # Add subtitle to title to ensure uniqueness (e.g. "GDP > 3%")
                    full_title = f"{event.get('title')} - {m.get('yes_sub_title', '')}"
                    
                    all_rows.append({
                        "Platform": "Kalshi",
                        "Title": full_title,
                        "Outcomes": row_outcomes
                    })
            
            if not cursor: break
            if not FETCH_ALL_PAGES and len(all_rows) >= 200: break
            time.sleep(0.1)
            
        except Exception as e:
            print(f"   ⚠️ Kalshi Error: {e}")
            break
            
    print(f"   ✅ Loaded {len(all_rows)} Kalshi events.")
    return pd.DataFrame(all_rows)

# --- PART 2: EXECUTION ---

# 1. Fetch Data
df_p = get_polymarket_data()
df_k = get_kalshi_data()

# 2. Filter for Binary Markets (Strictly 2 outcomes)
print("  Filtering for binary markets...")
p_binary = df_p[df_p['Outcomes'].apply(lambda x: len(x) == 2)].reset_index(drop=True)
k_binary = df_k[df_k['Outcomes'].apply(lambda x: len(x) == 2)].reset_index(drop=True)

# 3. Vectorize Titles
print(" [3/3] Running AI Vector Search (all-MiniLM-L6-v2)...")
model = SentenceTransformer('all-MiniLM-L6-v2')
p_vecs = model.encode(p_binary['Title'].tolist(), show_progress_bar=False)
k_vecs = model.encode(k_binary['Title'].tolist(), show_progress_bar=False)

# 4. Find Arbitrage
print(" Scanning for VERIFIED profit opportunities...")
sim_matrix = cosine_similarity(k_vecs, p_vecs)
opportunities = []

for k_idx, row in enumerate(sim_matrix):
    best_p_idx = np.argmax(row)
    score = row[best_p_idx]
    
    # Threshold lowered slightly to catch "Week 10" vs "Week 10." matches
    if score > 0.82: 
        k_row = k_binary.iloc[k_idx]
        p_row = p_binary.iloc[best_p_idx]
        
        # A. STRICT NUMBER GUARD ---
        # Filters out "GDP > 3%" vs "GDP > 6%" mismatches
        p_nums = extract_numbers(p_row['Title'])
        k_nums = extract_numbers(k_row['Title'])
        
        # If both titles have numbers, they MUST overlap.
        # (Empty intersection implies mismatch).
        if p_nums and k_nums and not p_nums.intersection(k_nums):
            continue 

        #  OUTCOME & PRICE LOGIC ---
        try:
            p_outs = p_row['Outcomes']
            k_outs = k_row['Outcomes']
            
            # Identify Keys: Look for "Yes", "Trump", "Over", etc.
            p_yes = next(k for k in p_outs if "Yes" in k or "Trump" in k or "Over" in k)
            p_no = next(k for k in p_outs if k != p_yes)
            
            # Kalshi is standardized in our fetch function
            k_yes = "Yes"
            k_no = "No"
            
            # Extract Prices
            py = p_outs[p_yes]
            pn = p_outs[p_no]
            ky = k_outs[k_yes]
            kn = k_outs[k_no]
            
            # Calculate Total Cost for both Hedge Strategies
            cost_A = py + kn # Buy Poly YES + Kalshi NO
            cost_B = pn + ky # Buy Poly NO + Kalshi YES
            
            # Find best strategy
            min_cost = min(cost_A, cost_B)
            implied_profit = (1.0 - min_cost) * 100
            
            # PROFIT FILTER ---
            # Real arb is rarely > 20%. Anything higher is likely a data error.
            if PROFIT_MIN < implied_profit < PROFIT_MAX:
                opportunities.append({
                    "Event": p_row['Title'],
                    "Strategy": "Buy Poly YES / Kalshi NO" if cost_A < cost_B else "Buy Poly NO / Kalshi YES",
                    "Total_Cost": f"${min_cost:.3f}",
                    "Profit": f"{implied_profit:.1f}%",
                    "Score": f"{score:.2f}",
                    "Poly_Title": p_row['Title'],   # For manual verification
                    "Kalshi_Title": k_row['Title']
                })
        except: continue

# 5. Output Results
final_df = pd.DataFrame(opportunities)

if not final_df.empty:
    print(f"\n✅ SUCCESS: Found {len(final_df)} verified opportunities!")
    
    # Sort by highest profit
    display_cols = ['Event', 'Profit', 'Total_Cost', 'Strategy', 'Poly_Title', 'Kalshi_Title']
    display(final_df[display_cols].sort_values("Profit", ascending=False).head(15))
    
    # Save to CSV
    final_df.to_csv("arbitrage_opportunities.csv", index=False)
    print("\n📁 Saved results to 'arbitrage_opportunities.csv'")
else:
    print("\nNo risk-free arbitrage found within the 0.5% - 20% verified range.")

c:\Users\lucam\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🔵 [1/3] Fetching Polymarket Data...
   ✅ Loaded 29811 Polymarket events.
🟢 [2/3] Fetching Kalshi Data...
   ✅ Loaded 30920 Kalshi events.
  Filtering for binary markets...
 [3/3] Running AI Vector Search (all-MiniLM-L6-v2)...


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 864.64it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


 Scanning for VERIFIED profit opportunities...

✅ SUCCESS: Found 1066 verified opportunities!


,Event,Profit,Total_Cost,Strategy,Poly_Title,Kalshi_Title
529,US GDP growth in Q1 2026?,9.9%,$0.901,Buy Poly YES / Kalshi NO,US GDP growth in Q1 2026?,US GDP growth in Q1 2026? - Above 3.5%
241,Which company's AI will first hit 1500 on Chat...,9.8%,$0.902,Buy Poly YES / Kalshi NO,Which company's AI will first hit 1500 on Chat...,Which AI will be the first to hit 1500 on Chat...
98,GDP growth in 2026,9.8%,$0.902,Buy Poly NO / Kalshi YES,GDP growth in 2026,GDP growth in 2026? - 5.6 to 6.0
354,How many Republican Senators not running in 2026?,9.7%,$0.903,Buy Poly YES / Kalshi NO,How many Republican Senators not running in 2026?,How many Republican senators will lose reelect...
392,Alaska Governor Election Winner,9.7%,$0.903,Buy Poly NO / Kalshi YES,Alaska Governor Election Winner,Alaska Governor winner? (Person) - Matthew Claman
142,Who will be the next Secretary-General of the UN?,9.6%,$0.904,Buy Poly YES / Kalshi NO,Who will be the next Secretary-General of the UN?,Who will be the next UN Secretary-General? - M...
780,SSC Napoli vs. AS Roma,9.5%,$0.905,Buy Poly NO / Kalshi YES,SSC Napoli vs. AS Roma,Napoli vs Roma - Tie
813,Ethereum price on February 13?,9.5%,$0.905,Buy Poly YES / Kalshi NO,Ethereum price on February 13?,"Ethereum price on Feb 13, 2026 at 5pm EST? - $..."
812,Ethereum price on February 13?,9.5%,$0.905,Buy Poly YES / Kalshi NO,Ethereum price on February 13?,"Ethereum price on Feb 13, 2026 at 5pm EST? - $..."
798,Ethereum price on February 13?,9.5%,$0.905,Buy Poly YES / Kalshi NO,Ethereum price on February 13?,"Ethereum price on Feb 13, 2026 at 5pm EST? - $..."



📁 Saved results to 'arbitrage_opportunities.csv'
